In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end

#####################################


(-3.9344419569678464+0j) [] +
(-0.04069607268303363+0j) [X0 X1 X2] +
(0.003136942278337758+0j) [X0 X1 X2 X3 Y7 Y11] +
(-0.0016354065601825191+0j) [X0 X1 X2 Y3 Y5] +
(-0.003714482527051004+0j) [X0 X1 X2 Z3] +
(-0.001780970375182675+0j) [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
(-0.004791170475023733+0j) [X0 X1 Z2 Y3 Y4 X5] +
(0.0012919456388330267+0j) [X0 X1 X3 X4 Y7 Y11] +
(-0.0010954854746108838+0j) [X0 X1 X3 Y4 Y5 Z6 Z7] +
(-0.0012919456388330267+0j) [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
(-0.0025623897800114777+0j) [X0 X1 X3 Z4 Y5 Y6 Z7] +
(-8.04911530597452e-05+0j) [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
(-0.0013724367918927717+0j) [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
(-0.0009084689616229715+0j) [X0 X1 X3 X6 Y7 Y11] +
(8.981653103288107e-05+0j) [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
(-0.0009084689616229714+0j) [X0 X1 X3 Y7 X8 Y11] +
(0.0009084689616229714+0j) [X0 X1 X3 Y7 Y8 Z10 X11] +
(8.981653103288189e-05+0j) [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
(-0.0008186524305900897+0j) [X0 X1 X3 Y7 Z8 Y10 X11] +
(-0.017297109487009115+0j)

In [3]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    occ_threshold= 1.999,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)
reduced_Sec_Quant_CC_ops_ijab

[-1.0 [2^ 3^ 4 5] +
 1.0 [5^ 4^ 3 2],
 -1.0 [2^ 3^ 4 7] +
 1.0 [7^ 4^ 3 2],
 -1.0 [2^ 3^ 4 9] +
 1.0 [9^ 4^ 3 2],
 -1.0 [2^ 3^ 6 7] +
 1.0 [7^ 6^ 3 2],
 -1.0 [2^ 3^ 6 9] +
 1.0 [9^ 6^ 3 2],
 -1.0 [2^ 3^ 8 9] +
 1.0 [9^ 8^ 3 2]]

In [6]:
# note transform set to 'BK'
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia, 
                                                                                                                reduced_Sec_Quant_CC_ops_ijab,
                                                                                                                   transformation='BK')

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [14]:
new_BK = BK_Qubit_Reordering(
                QubitHamiltonian,
                 Qubit_Op_list_Second_Quant_CC_Ops_ia,
                 Qubit_Op_list_Second_Quant_CC_Ops_ijab,
                 Hamilt.molecule.n_qubits,
                ansatz_obj.Get_BK_HF_state_in_OCC_basis(),
                Hamilt.molecule.n_electrons)

REDUCED_Qubit_MolecularHamiltonian = new_BK.Get_Reordered_Hamiltonian_2_qubits_removed()

In [15]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(REDUCED_Qubit_MolecularHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 602/602 [00:15<00:00, 39.77it/s] 


{0: [(-3.8496909545430444+0j) []],
 1: [(0.18817606710188547+0j) [Z9]],
 2: [(-0.21675429325000517+0j) [Z8 Z9],
  (0.004217284878422763+0j) [Y6 Y8 Z9],
  (0.010590591491110787+0j) [X4 X8 Z9],
  (-0.0025623897800114777+0j) [Z0 X1 Y3 Y4 X8 Z9],
  (0.004889539651547374+0j) [Z1 Z2 Y3 Y4 X8 Z9],
  (0.003049883009254853+0j) [Z3 Y4 Y7 X8 Z10]],
 3: [(-0.2167542932500052+0j) [Z8],
  (0.004217284878422763+0j) [X6 X8 Z9],
  (0.010590591491110787+0j) [Y4 Y8],
  (0.0025623897800114777+0j) [Z0 X1 Y3 X4 Y8 Z9],
  (-0.004889539651547374+0j) [Z1 Z2 Y3 X4 Y8 Z9],
  (-0.003049883009254853+0j) [Z3 X4 Y7 Y8 Z10]],
 4: [(-0.2167542932500054+0j) [Z6],
  (0.010590591491110787+0j) [Z3 X4 X6 Z7],
  (0.003832986409513871+0j) [Y6 Z9 Y10],
  (0.0030498830092548536+0j) [Z4 Y6 X7 Z9 X10],
  (0.003049883009254853+0j) [Z3 X4 X6 Y7],
  (0.0009084689616229715+0j) [X0 X1 Y3 Y6 X7 Z9 Z10],
  (-0.004031651676556212+0j) [Z1 X2 Y3 Y6 X7 Z9 Z10]],
 5: [(0.0698018080330069+0j) [Z6 Z8 Z9],
  (0.010590591491110787+0j) [Y4 Y8 Z9

In [20]:
Qubit_Op_list_Second_Quant_CC_Ops_ijab

[0.125j [Z1 X2 Z3 Y4] +
 0.125j [Z1 X2 Z3 Y4 Z5] +
 -0.125j [Z1 Y2 Z3 X4] +
 -0.125j [Z1 Y2 Z3 X4 Z5] +
 0.125j [X2 Y4] +
 0.125j [X2 Y4 Z5] +
 -0.125j [Y2 X4] +
 -0.125j [Y2 X4 Z5],
 0.125j [Z1 X2 Z3 X4 Y5 Z6] +
 0.125j [Z1 X2 Y4 X5 Z7] +
 0.125j [Z1 Y2 Z3 Y4 Y5 Z6] +
 -0.125j [Z1 Y2 X4 X5 Z7] +
 0.125j [X2 Z3 Y4 X5 Z7] +
 0.125j [X2 X4 Y5 Z6] +
 -0.125j [Y2 Z3 X4 X5 Z7] +
 0.125j [Y2 Y4 Y5 Z6],
 0.125j [Z1 X2 X4 X5 Y7 Z8 X9] +
 -0.125j [Z1 X2 Y4 X5 Y7 Y9] +
 0.125j [Z1 Y2 X4 X5 Y7 Y9] +
 0.125j [Z1 Y2 Y4 X5 Y7 Z8 X9] +
 0.125j [X2 Z3 X4 X5 Y7 Z8 X9] +
 -0.125j [X2 Z3 Y4 X5 Y7 Y9] +
 0.125j [Y2 Z3 X4 X5 Y7 Y9] +
 0.125j [Y2 Z3 Y4 X5 Y7 Z8 X9],
 0.125j [Z1 X2 Z3 Y6] +
 0.125j [Z1 X2 Z5 Y6 Z7] +
 -0.125j [Z1 Y2 Z3 X6] +
 -0.125j [Z1 Y2 Z5 X6 Z7] +
 0.125j [X2 Z3 Z5 Y6 Z7] +
 0.125j [X2 Y6] +
 -0.125j [Y2 Z3 Z5 X6 Z7] +
 -0.125j [Y2 X6],
 0.125j [Z1 X2 Z5 X6 Y7 Z8 X9] +
 -0.125j [Z1 X2 Z5 Y6 Y7 Y9] +
 0.125j [Z1 Y2 Z5 X6 Y7 Y9] +
 0.125j [Z1 Y2 Z5 Y6 Y7 Z8 X9] +
 0.125j [X2 Z3 Z5 X6 Y7 Z

In [18]:
new_BK.Get_Reordered_CC_qubit_terms_2_qubits_removed()

[0.25j [Z1 X2 Z3 Y4] +
 -0.25j [Z1 Y2 Z3 X4] +
 0.25j [X2 Y4] +
 -0.25j [Y2 X4],
 0.125j [Z1 X2 Z3 X4 Z6] +
 0.125j [Z1 X2 Y4 Z7] +
 0.125j [Z1 Y2 Z3 Y4 Z6] +
 -0.125j [Z1 Y2 X4 Z7] +
 0.125j [X2 Z3 Y4 Z7] +
 0.125j [X2 X4 Z6] +
 -0.125j [Y2 Z3 X4 Z7] +
 0.125j [Y2 Y4 Z6],
 0.125j [Z1 X2 X4 Y7 Z8 X9] +
 -0.125j [Z1 X2 Y4 Y7 Y9] +
 0.125j [Z1 Y2 X4 Y7 Y9] +
 0.125j [Z1 Y2 Y4 Y7 Z8 X9] +
 0.125j [X2 Z3 X4 Y7 Z8 X9] +
 -0.125j [X2 Z3 Y4 Y7 Y9] +
 0.125j [Y2 Z3 X4 Y7 Y9] +
 0.125j [Y2 Z3 Y4 Y7 Z8 X9],
 0.125j [Z1 X2 Z3 Y6] +
 0.125j [Z1 X2 Y6 Z7] +
 -0.125j [Z1 Y2 Z3 X6] +
 -0.125j [Z1 Y2 X6 Z7] +
 0.125j [X2 Z3 Y6 Z7] +
 0.125j [X2 Y6] +
 -0.125j [Y2 Z3 X6 Z7] +
 -0.125j [Y2 X6],
 0.125j [Z1 X2 X6 Y7 Z8 X9] +
 -0.125j [Z1 X2 Y6 Y7 Y9] +
 0.125j [Z1 Y2 X6 Y7 Y9] +
 0.125j [Z1 Y2 Y6 Y7 Z8 X9] +
 0.125j [X2 Z3 X6 Y7 Z8 X9] +
 -0.125j [X2 Z3 Y6 Y7 Y9] +
 0.125j [Y2 Z3 X6 Y7 Y9] +
 0.125j [Y2 Z3 Y6 Y7 Z8 X9],
 0.125j [Z1 X2 Z3 Y8] +
 0.125j [Z1 X2 Z3 Y8 Z9] +
 -0.125j [Z1 Y2 Z3 X8] +
 -0.125j 

In [ ]:
from quchem.Simulating_Quantum_Circuit import *

In [ ]:
n_shots= 1000


def VQE_experiment_ENERGY(theta_ia_ijab_combined_list):
    theta_params_ia = [theta_ia_ijab_combined_list[0], theta_ia_ijab_combined_list[1]]
    theta_params_ijab = [theta_ia_ijab_combined_list[2]]
    
    
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_params_ia, 
                                                              theta_params_ijab)
    

    VQE_exp = VQE_Experiment(QubitHamiltonian, ansatz_cirq_circuit, n_shots)
    return VQE_exp.Calc_Energy().real

In [ ]:
theta_combined = [1,2, np.pi]

VQE_experiment_ENERGY(theta_combined)

In [ ]:
import random

theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ijab))]

theta_combined_random_input = [*theta_ia_random_input, *theta_ijab_random_input]

In [ ]:
### optimizer
from quchem.Scipy_Optimizer import *

GG = Optimizer(VQE_experiment_ENERGY, theta_combined_random_input, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
Hamilt.molecule.fci_energy

In [ ]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle -\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [ ]:
def calc_gradient(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = VQE_experiment_ENERGY(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = VQE_experiment_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append((gradient, theta))
        

    return grad_list

note:
this is very SLOW as it has to run a separate experiment TWICE for each parameter before taking a step!

In [ ]:
X0 = [random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ia) + len(Sec_Quant_CC_ops_ijab))]

tf_opt = Tensor_Flow_Optimizer(VQE_experiment_ENERGY, X0, 'Adam', calc_gradient, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
tf_opt.optimize(50)
tf_opt.plot_convergence()

In [ ]:
from quchem.Adam_Optimizer import *

In [ ]:
def calc_gradient_ADAM(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = VQE_experiment_ENERGY(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = VQE_experiment_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append(gradient)
        

    return np.array(grad_list)

In [ ]:
X0 = np.array([random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ia) + len(Sec_Quant_CC_ops_ijab))])

opt_params, list_of_inputs, list_of_outputs = Adam_Opt(X0, VQE_experiment_ENERGY,
                                                       calc_gradient_ADAM, 
                                                       learning_rate=0.1, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-3,
                                                       store_steps=True)

VQE_experiment_ENERGY(opt_params)

In [ ]:
import matplotlib.pyplot as plt
# % matplotlib inline
plt.figure()
plt.plot(list_of_outputs)
plt.xlabel('iterations')
plt.ylabel('objective function value')
plt.show()